<h1> Visualizing Pretrained BERT embeddings in Tensorboard </h1>

In [1]:
# ( Optional) 
# The embeddings were first serialized on disk and then loaded back using pickle
import pickle

def save_to_disk(path, objs):
  # Saving the objects:
  with open(path, 'wb') as f:
      pickle.dump(objs, f)

def load_from_disk(path):
  # Getting back the objects:
  with open(path, "rb") as f:
    return pickle.load(f)

In [2]:
base_path = "./emb_data"
path = base_path + "/embeddings.pkl"
path_lbls = base_path + "/emb_labels.pkl"

embeddings =  load_from_disk(path)[0]
emb_labels = load_from_disk(path_lbls)[0]
# Here embeddings are a 2D array/tensor, lables can be strings or numeric 
embeddings.shape, emb_labels.shape

((3744, 1024), (3744,))

In [6]:
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector

sess = tf.InteractiveSession()

with tf.device("/cpu:0"):
    # Pass 2D array-embeddings to createa a tensor     
    tf_embedding = tf.Variable(embeddings, trainable = False, name = "embedding")

max_size = embeddings.shape[0]
tf.global_variables_initializer().run()
# Output path to save weights
path = "./tensorboard"
saver = tf.train.Saver()
writer = tf.summary.FileWriter(path, sess.graph)
config = projector.ProjectorConfig()
embed = config.embeddings.add()
embed.tensor_name = "embedding"
# This file needs to be there on disk to pick the corresponding labels against each embedding
embed.metadata_path = "metadata.tsv"
projector.visualize_embeddings(writer, config)
# Save tensorflow embeddings as a model checkpoint
saver.save(sess, path +'/model.ckpt' , global_step=max_size)
sess.close()

<h3>Creating Labels ".tsv" file</h3>

In [7]:
# Open a file with access mode 'a' i.e., append
with open("./tensorboard/metadata.tsv", "a") as file_object:
    for x in emb_labels:
        file_object.write(x + "\n")

<h2>Optionally redirect ngork to expose local tensorboard visualization to public </h2>

In [8]:
get_ipython().system_raw('./ngrok http 8080 &')

In [8]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"